In [2]:
!pip install boto3 -q

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 139.6/139.6 kB 4.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 13.5/13.5 MB 75.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 84.4/84.4 kB 8.4 MB/s eta 0:00:00


In [3]:
import boto3
import json
from google.colab import userdata
aws_access_key_id = userdata.get('aws_access_key_id')
aws_secret_key = userdata.get('aws_secret_key')

In [4]:
# Initialize Bedrock client
bedrock_agent_client = boto3.client(
    "bedrock-agent",
    region_name="us-east-2",
    aws_access_key_id=aws_access_key_id,
    aws_secret_access_key=aws_secret_key,
    )

# Define the agent name and description
agent_name = "CustomerSupportAgent"
agent_description = "A chatbot for assisting customers with different tasks."

In [5]:
agent_instruction = '''
You are a friendly and helpful customer service associate working at an outdoor cloth e-commerse company called 'Backcountry'. Your name is Dennis.

Your task is to help customers with inquries on products. Briefly introduce yourself and your task at the beginning of the conversation and ask for customer's information [Full Name, Gender, Age, Activities] to better assist them. After customer reply, use the tool provided to store the info to S3 bucket. Then ask 'Thanks for providing the info! How can I help you today?'. If customer ignore the message or choose to not share the info, do not ask again.

When customer ask for recommendation, search the knowledge base try to give 2 recommendation with details in bullet points for customer to choose. If you cannot find the product, apologies to the customer and state that we don't currently sell this type of product.

'''



In [7]:
# Create the agent
response = bedrock_agent_client.create_agent(
    agentName=agent_name,
    description=agent_description,
    instruction=agent_instruction,
    foundationModel="anthropic.claude-3-7-sonnet-20250219-v1:0",
    idleSessionTTLInSeconds=300  # Set session timeout (5 minutes)
)

# Print the response
print("Agent Created:", response)



Agent Created: {'ResponseMetadata': {'RequestId': '7e01c660-c38a-41cf-b368-dbd075b5b0dd', 'HTTPStatusCode': 202, 'HTTPHeaders': {'date': 'Thu, 03 Apr 2025 05:47:15 GMT', 'content-type': 'application/json', 'content-length': '1379', 'connection': 'keep-alive', 'x-amzn-requestid': '7e01c660-c38a-41cf-b368-dbd075b5b0dd', 'x-amz-apigw-id': 'IboTkG0qCYcEYhQ=', 'x-amzn-trace-id': 'Root=1-67ee20e3-1a0e253f4555a23c60e40d4e'}, 'RetryAttempts': 0}, 'agent': {'agentArn': 'arn:aws:bedrock:us-east-2:324037274971:agent/LPHJ94LMYT', 'agentCollaboration': 'DISABLED', 'agentId': 'LPHJ94LMYT', 'agentName': 'CustomerSupportAgent', 'agentStatus': 'CREATING', 'createdAt': datetime.datetime(2025, 4, 3, 5, 47, 15, 308502, tzinfo=tzlocal()), 'description': 'A chatbot for assisting customers with different tasks.', 'foundationModel': 'anthropic.claude-3-7-sonnet-20250219-v1:0', 'idleSessionTTLInSeconds': 300, 'instruction': "\nYou are a friendly and helpful customer service associate working at an outdoor clot